# Objectives
The purpose of this section is to implement a dictionary attack over a WPA2-Personal wifi.To answer most of the following questions, we used the Python cryptography library.

# Preliminaries
File tradio.pcapng is a capture test that contains a WPA2-Personal handshake and some ping packets after handshake for the following wifi session:

*   SSID: Wifi Test
*   Password: Wifi Test Password

You can identify the 4 handshake messages by applying filter eapol in wireshark. Also, you can
decrypt traffic (see: https://wiki.wireshark.org/HowToDecrypt802.11).

# Question 1
For WPA2 personal, PMK is PSK. Prove that PMK is: a22c...b603

The first thing we have to do is to derive the PMK or also called PSK from:


*   SSID
*   Password




In [21]:
from cryptography.hazmat.primitives.kdf.pbkdf2  import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes

ssid = "Wifi_Test"  # SSID (Wi-Fi network name)
password = "Wifi_Test_Password"     # Wi-Fi password

# Generate salt based on the SSID
salt = ssid.encode()

# Define parameters for PBKDF2 key derivation function
key_length = 32    # Length of the derived key (PMK) in bytes
iterations = 4096  # Number of iterations

# Derive PMK using PBKDF2HMAC key derivation function
kdf = PBKDF2HMAC(
        algorithm=hashes.SHA1(),
        length=key_length,
        salt=salt,
        iterations=iterations,
)
pmk = kdf.derive(password.encode())

# Convert PMK bytes to hexadecimal representation
pmk_hex = pmk.hex()
print("Derived PMK:", pmk_hex)

Derived PMK: a22c4740d1843aca77252e5a6923af13773be449bb443edca5c6296ca151b603


# Question 2.
 Parse the capture file 'tradio.pcapng' with tshark and dump the required handshake packets in hexadecimal. From the first handshake message, prove that nonce is b6ea...d7d3b. From the second handshake message, prove that nonce is e6d8...b61c.
